In [1]:
import numpy as np
from tqdm import tqdm
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from utils import print_confusion_matrix, assemble_dataset_supervised_learning
from sklearn.ensemble import VotingClassifier

## load data

In [ ]:
main_path = r"./" #path were the data is stored

peaklist = np.array(pd.read_csv(r'.\\regions_peaklist.txt', sep = " ")) #load the preselected of peaklist



path_data = r'.\msi_tables_filtered'
list_dataset = os.listdir(path_data)




labels = pd.read_csv(os.path.join(main_path,'labels_frozen.txt'),sep = ';' )

full_dataset, y_labels = assemble_dataset_supervised_learning(labels,list_dataset,path_data, data_type = "stroma")

## pre-process data per patient with box cox and 10**5 factor

In [ ]:
dict_X_gauss = {}

pt = preprocessing.PowerTransformer(method='box-cox', standardize=False)
name_images = full_dataset[full_dataset["dataset_name"]=="SlideA1"]["image_name"]
temp_patient_data = full_dataset[full_dataset["dataset_name"]=="SlideA1"].drop(columns = ['dataset_name','image_name'])*10**5
X_gaus = pt.fit_transform(temp_patient_data)

columns = np.unique(full_dataset["dataset_name"])

for col in tqdm( columns[1:]):
    name_images = full_dataset[full_dataset["dataset_name"]==col]["image_name"]
    temp_patient_data = full_dataset[full_dataset["dataset_name"]==col].drop(columns = ['dataset_name','image_name'])*10**5
    array_trans = pt.fit_transform(temp_patient_data)
    X_gaus=np.concatenate((X_gaus,array_trans),axis =0)

In [ ]:
X_train, X_test_and_valid, y_train, y_test_and_valid, data_train, data_test_and_valid = train_test_split(X_gaus,y_labels , full_dataset[["dataset_name",'image_name']],test_size = 0.30, random_state=10) 

In [ ]:
#create test dataset
len_half = len(y_test_and_valid)//2
X_test = X_test_and_valid[:len_half]
data_test = data_test_and_valid[:len_half]
y_test = y_test_and_valid[:len_half]

In [ ]:
#create validation dataset
X_valid = X_test_and_valid[len_half:]
data_valid = data_test_and_valid[len_half:]
y_valid = y_test_and_valid[len_half:]

In [ ]:
##saving data train, test and valid to reuse in H&E pipeline

#data_train.insert(loc=1, column='labels', value=y_train)
#data_test.insert(loc=1, column='labels', value=y_test)
#data_valid.insert(loc=1, column='labels', value=y_valid)

#
#data_train.to_csv('data_train_stroma_vs_epithelial_tissue.csv',index=False)
#data_test.to_csv('data_test_stroma_vs_epithelial_tissue.csv',index=False)
#data_valid.to_csv('data_valid_stroma_vs_epithelial_tissue.csv',index=False)

In [ ]:
data = pd.read_csv(os.path.join(main_path,'data_train_stroma_vs_epithelial_tissue.csv'),sep = ',' )

X_train = []
y_train = []
train_paths = []
for slide in tqdm(os.listdir(os.path.join(main_path, 'Slides'))):
    tile_path = os.path.join(main_path, 'Slides',slide,'tiles')
    gland = data[(data['labels']=='stroma') & (data['dataset_name']==slide)]['image_name']
    tissue = data[(data['labels']== "epithelial tissue") & (data['dataset_name']==slide)]['image_name']
    gland = list(gland)
    tissue = list(tissue)
    for image_path in gland:
        if os.path.isfile(os.path.join(tile_path, image_path)):
            X_train.append(dict_X_gauss[slide+image_path][0])
            y_train.append("stroma")
            train_paths.append(os.path.join(tile_path, image_path))
        else:
            print("error for gland")
    for image_path in tissue:
        if os.path.isfile(os.path.join(tile_path, image_path)):
            X_train.append(dict_X_gauss[slide+image_path][0])
            y_train.append("epithelial tissue")
            train_paths.append(os.path.join(tile_path, image_path))

y_train = np.ravel(np.array(y_train))
            
df_features_train = pd.DataFrame(X_train,index= train_paths,columns=list(full_dataset.columns)[2:])
df_features_train["labels"]=y_train
#df_features_train.to_csv(r".\train_features_tissue_type_msi.csv")
            

In [ ]:
data = pd.read_csv(os.path.join(main_path,'data_test_stroma_vs_epithelial_tissue.csv'),sep = ',' )

X_test = []
y_test = []
test_paths = []
        
for slide in tqdm(os.listdir(os.path.join(main_path, 'Slides'))):
    tile_path = os.path.join(main_path, 'Slides',slide,'tiles')
    gland = data[(data['labels']=='stroma') & (data['dataset_name']==slide)]['image_name']
    tissue = data[(data['labels']=='epithelial tissue') & (data['dataset_name']==slide)]['image_name']
    gland = list(gland)
    tissue = list(tissue)
    for image_path in gland:
        if os.path.isfile(os.path.join(tile_path, image_path)):
            X_test.append(dict_X_gauss[slide+image_path][0])
            y_test.append("stroma")
            test_paths.append(os.path.join(tile_path, image_path))
    for image_path in tissue:
        if os.path.isfile(os.path.join(tile_path, image_path)):
            X_test.append(dict_X_gauss[slide+image_path][0])
            y_test.append("epithelial tissue")
            test_paths.append(os.path.join(tile_path, image_path))
            
y_test = np.ravel(np.array(y_test))

df_features_test = pd.DataFrame(X_test[:len(y_test)//2],index= test_paths[:len(y_test)//2],columns=list(full_dataset.columns)[2:])
df_features_test['labels']=y_test[:len(y_test)//2]
#df_features_test.to_csv(r".\test_features_tissue_type_msi.csv")

df_features_valid = pd.DataFrame(X_test[len(y_test)//2:],index= test_paths[len(y_test)//2:],columns=list(full_dataset.columns)[2:])
df_features_valid['labels']=y_test[len(y_test)//2:]
#df_features_valid.to_csv(r".\valid_features_tissue_type_msi.csv")



## gridsearchCV for MLPclassifier

In [ ]:
parameters = { 'batch_size':[32,64,128,356], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':list(product(np.arange(10,21,10),np.arange(10,21,10)))}
mlp_model = GridSearchCV(MLPClassifier(solver='adam',max_iter = 1100), parameters, n_jobs=20, , cv= 5, verbose = 2)

mlp_model.fit(X_train,y_train)

## gridsearchCV for random forest

In [ ]:
param_grid = { 
   'n_estimators': [100,200,500],
   'max_depth' : [4,8,16],
   'criterion' :['gini', 'entropy']
}

rf_model = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, verbose=1,n_jobs=20)
rf_model.fit(X_train, y_train)

## gridsearchCV for XGBoost

In [ ]:
params = {
   "min_child_weight":range(1,6,2),
   "gamma": uniform(0, 0.5),
   "learning_rate": uniform(0.03, 0.3),
   "max_depth": range(3,10,2), 
   "n_estimators": randint(100, 150),
   "subsample": uniform(0.6, 0.4)
}

xgb_model = GridSearchCV(estimator = xgb.XGBClassifier(colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = params, scoring='roc_auc',n_jobs=12,iid=False, cv=5,verbose=1)

xgb_model.fit(X_train, y_train)

## create feature importance file

In [ ]:
results=pd.DataFrame()
results['columns']=list(full_dataset.columns)[2:]
results['importances_rf'] = CV_rfc.feature_importances_
results['importances_xgboost'] = xgb1.feature_importances_
results.sort_values(by='importances_xgboost',ascending=False,inplace=True)
results.to_excel(r".\features_rf_xgboost_msi_gland_vs_tissue.xlsx",index=None)

## ensemble all best model

In [ ]:
vc = VotingClassifier(estimators=[
     ('mlp', mlp_model.best_estimator_), ('rf', rf_model.best_estimator_), ('xgb', xgb_model.best_estimator_)],
     voting='soft',n_jobs=12)
vc = vc.fit(np.array(X_train),np.array(y_train))

In [ ]:
predictions_test =vc.predict(np.array(X_test))
predictions_valid =vc.predict(np.array(X_valid))
predictions_train =vc.predict(np.array(X_train))

## classification report

In [ ]:
print(classification_report(y_train,predictions_train))  
print(classification_report(y_valid,predictions_valid))  
print(classification_report(y_test,predictions_test))  